In [1]:
import os
import numpy as np
import pandas as pd
import spacy
import re
from tqdm import tqdm

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
os.chdir('E:\[editted]\code\homework1')

In [4]:
###combine dataframe
dfs=[]
headers=['URL','Job Title','Company','Company URL','Company Location','Job Description']
for filename in os.scandir('job_info_data'):
  if filename.is_file():
        if '522' in filename.path:
            data=pd.read_csv(filename.path,delimiter=',')
            data.columns=headers #make headers match
            dfs.append(data)
            df=pd.concat(dfs)

In [5]:
df=df[df["Job Title"].notnull()]
df=df[df["Job Description"].notnull()]
df=df.drop_duplicates('URL').reset_index(drop=True)##not sure if need to drop duplicates but I drop them

Task 1. Create one pandas dataframe that combines all the data scraped from May 22, 2022 together. Drop rows with missing job titles and/or job descriptions. Use `spacy` to tokenize all the job titles included in the cleaned dataframe. For each job title, find all the nouns and all the adjectives in the title and get their lowercased lemmatized form. Use the reformatted nouns to construct a vocabulary set for this dataframe. How many unique nouns are there? Construct another vocabulary set using the reformatted adjectives. How many unique adjectives are there? What kind of different information do the nouns versus the adjectives reveal about the specific job? 

In [12]:
vocab_with_freq = dict()

for i in tqdm(range(len(df))):
    job_title = df.loc[i, 'Job Title']
    for token in nlp(job_title):
                if token.pos_=='NOUN' or token.pos_=='PROPN':
                    vocab_with_freq[token.lemma_.lower()] = vocab_with_freq.get(token.lemma_.lower(), 0) + 1

vocabulary_list1 = list(vocab_with_freq.keys())
print('there are', len(vocabulary_list1), 'unique nouns among indeed job titles')        

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:20<00:00, 172.92it/s]

there are 2100 unique nouns among indeed job titles


In [7]:
vocab_with_freq2 = dict()

for i in tqdm(range(len(df))):
    job_title = df.loc[i, 'Job Title']
    for token in nlp(job_title):
            if token.pos_=='ADJ':
                vocab_with_freq2[token.lemma_.lower()] = vocab_with_freq2.get(token.lemma_.lower(), 0) + 1

vocabulary_list2 = list(vocab_with_freq2.keys())
print('there are', len(vocabulary_list2), 'unique adjectives among indeed job titles')  

100%|██████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:35<00:00, 99.18it/s]

there are 84 unique adjectives among indeed job titles


Task 2. Choose the first job title in your dataframe as the primary string. Use one-hot encoding as the word embedding method and find jobs in your cleaned dataframe that have similar nouns in the title as your primary string. 

In [50]:
job_title = df.loc[0, 'Job Title'] 
token_indices = []

for token in nlp(job_title):
            if token.pos_=='NOUN'or token.pos_=='PROPN':
                 print(token.lemma_.lower())
          

pharmacy
technician
ii
-retail
pharmacy


In [44]:
one_hot_encodings = []
existent = []

for i in tqdm(range(len(df))):
    job_title = df.loc[i, 'Job Title'] 
    token_indices = []
    

    for token in nlp(job_title):
            if  token.pos_=='NOUN'or token.pos_=='PROPN':
                token_index_in_vocab = vocabulary_list1.index(token.lemma_.lower())
                token_indices.append(token_index_in_vocab)

    one_hot_encoding = np.zeros(len(vocabulary_list1))
    if token_indices:
        existent.append(i)
    for token_index in token_indices:
        one_hot_encoding[token_index] = 1

    one_hot_encodings.append(one_hot_encoding)

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:18<00:00, 192.49it/s]


In [46]:
from scipy.spatial.distance import cosine

title_a = df.loc[0, 'Job Title']
similarity_values = []
for i in range(1, len(df)):
    if i in existent:
        similarity_value = 1 - cosine(one_hot_encodings[0], one_hot_encodings[i])
        similarity_values.append(similarity_value)
    else:
        similarity_values.append(np.nan)

In [47]:
similar_df = pd.DataFrame(columns=['job_title', 'similarity_value_with_one_hot'])
similar_df['job_title'] = df.loc[1:, 'Job Title']
similar_df['similarity_value_with_one_hot'] = similarity_values

similar_df.sort_values(by='similarity_value_with_one_hot', ascending=False)
similar_df.nlargest(60, 'similarity_value_with_one_hot')

,job_title,similarity_value_with_one_hot
437,Pharmacy Technician,0.707107
471,Pharmacy Technician,0.707107
774,Pharmacy Technician,0.707107
800,Pharmacy Technician,0.707107
824,Pharmacy Technician,0.707107
1006,Nationally Certified Pharmacy Technician,0.707107
1858,Pharmacy Technician,0.707107
1908,Pharmacy Technician,0.707107
2269,Pharmacy Technician,0.707107
2907,Pharmacy Technician,0.707107


word vector

In [24]:
job_title = df.loc[0, 'Job Title'] 
token_indices = []

for token in nlp(job_title): 
     if token.is_alpha:
        if token.pos_=='NOUN':
            print(token.lemma_.lower())
            print(token.vector, len(token.vector))

pharmacy
[ 0.5329317   0.160685   -0.5223515   0.80456847 -0.34936082  0.691764
 -1.3658403   0.31263214  0.5571234   0.08278453  1.3027737  -0.09548701
  0.26102644  0.33520383 -0.59733295  0.33713377  1.5453093  -0.07921885
  0.6418117  -0.42891258  0.2758968  -0.984888   -0.360137   -0.7650051
  0.9531846   0.5456793  -0.26387167  1.5937123   0.5114108   0.83167154
  0.36495885  0.04516274 -1.015482   -0.45563123  0.18730639 -1.0458035
  0.16175903  0.28529122 -0.90085614  1.4278584  -0.6499713   0.17930278
 -0.74243826 -1.302859    1.5725551  -0.36354285  0.5514792  -0.01359016
 -1.196868   -1.5602034   0.34736043  0.42733163 -0.43997997 -1.3551067
  1.0745281  -0.03383554  0.9676018  -0.5148288  -1.4490203   0.12455095
 -0.9580579   0.44441816  1.6824095   0.0858326  -0.08211006 -0.7378588
 -0.24343312  0.7857573   0.14963377 -0.6974355   0.10117382  0.46376204
 -0.4656161  -0.8130659  -0.37113512  1.5239341  -0.349994    0.6277942
 -0.38428846  1.54196     0.21696147 -1.1119186  

In [31]:
spacy_encoding = np.array([float(0) for i in range(96)])
token_count = 0

for i in tqdm(range(len(df))):
    job_title = df.loc[i, 'Job Title'] 

    for token in nlp(job_title): 
         if token.is_alpha:
                if token.pos_=='NOUN' or token.pos_=='PROPN':
                    token_count += 1
                    spacy_encoding += np.array(token.vector)
spacy_encoding /= token_count

spacy_encoding

100%|█████████████████████████████████████████████████████████████████████████████| 3554/3554 [00:16<00:00, 212.25it/s]


array([ 1.05149723,  0.14177672, -0.09876894, -0.35119926, -0.06413789,
        0.47125474, -0.69683404,  0.27803691,  0.35268057, -0.31381226,
        0.8076313 , -0.05604623,  0.03855227, -0.11629709, -0.14551855,
       -0.16227251,  1.04469741,  0.31638261,  0.27688566, -0.34872296,
       -0.39789943, -0.76632548, -0.16971439,  0.16475175,  0.8245986 ,
        0.22984736, -0.39001606,  0.7191471 , -0.19318833,  0.9171662 ,
        0.02479396,  0.03144033, -0.69566317, -0.518414  ,  0.60410791,
        0.08652811,  0.07740026,  0.2323642 , -0.65052209, -0.05213718,
       -0.4262901 ,  0.08830329, -0.60387487, -0.71807706,  0.41031208,
       -1.16764318,  0.45469659,  0.76323245,  0.08471907, -0.61504198,
        0.18325466, -0.1419342 , -0.36760237, -1.3683226 ,  0.12273322,
        0.65018587,  0.8397558 , -0.44555939, -0.79092513,  0.20245137,
       -0.38333938, -0.42688764,  0.64808681, -0.61626891,  0.70970928,
       -0.81654168,  0.01532306,  0.00690288,  0.16866223, -0.91